In [ ]:
import pandas as pd
import numpy as np
from tslearn.neighbors import KNeighborsTimeSeries
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from tslearn.svm import TimeSeriesSVC
import shap
import wandb
import tsai.all
import imblearn
from tqdm import tqdm
import time
from IPython.display import clear_output
import logging
from datetime import datetime
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [ ]:
logging.basicConfig(filename = f'logs/ts_coin_ts_shap_generate_output_{datetime.now():%Y_%m_%d_%H_%M}.log',
                    level = logging.INFO,
                    format = '%(asctime)s:%(levelname)s:%(name)s:%(message)s')

In [ ]:
%run ../shared_functions.py
%run ../my_shared_functions.py

DIR_INPUT = '../../fraud-detection-handbook/simulated-data-transformed/data/'
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, "2018-06-11", END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),
                                                                    transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"
input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

BEGIN_DATE = "2018-08-08"
start_date_training = datetime.datetime.strptime(BEGIN_DATE, "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7
delta_valid = delta_test

(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training,
                            delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)

SEQ_LEN = 5

# By default, scales input data
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

SEED = 42
seed_everything(SEED)

In [ ]:
models = [
    { # 0
        'artifact_type': 'cnn',
        'artifact_name': 'cnn:v1',
        'artifact_path': 'cnn-v1/cnn_model.pt',
        'model_instance': FraudConvNet(len(input_features), SEQ_LEN).to(DEVICE),
        'model_type': 'handbook'
    },
    { # 1
        'artifact_type': 'lstm',
        'artifact_name': 'lstm:v1',
        'artifact_path': 'lstm-v1/lstm_model.pt',
        'model_instance': FraudLSTM(len(input_features)).to(DEVICE),
        'model_type': 'handbook'
    },
    { # 2
        'artifact_type': 'lstm',
        'artifact_name': 'lstm_attention:v1',
        'artifact_path': 'lstm_attention-v1/lstm_attention_model.pt',
        'model_instance': FraudLSTMWithAttention(len(input_features)).to(DEVICE),
        'model_type': 'handbook'
    },
    { # 3
        'artifact_type': 'cnn',
        'artifact_name': 'cnn_hypertuned:v1',
        'artifact_path': 'cnn_hypertuned-v1/cnn_hypertuned_model.pt',
        'model_instance': FraudConvNetWithDropout(len(input_features), hidden_size=500,
                                                  conv2_params=(100,2), p=0.2).to(DEVICE),
        'model_type': 'handbook'    
    },
    { # 4
        'artifact_type': 'lstm',
        'artifact_name': 'lstm_hypertuned:v1',
        'artifact_path': 'lstm_hypertuned-v1/lstm_hypertuned_model.pt',
        'model_instance': FraudLSTM(len(input_features), hidden_size=500, dropout_lstm=0.2).to(DEVICE),
        'model_type': 'handbook'
    },
    { # 5
        'artifact_type': 'lstm',
        'artifact_name': 'lstm_attention_hypertuned:v1',
        'artifact_path': 'lstm_attention_hypertuned-v1/lstm_attention_hypertuned_model.pt',
        'model_instance': FraudLSTMWithAttention(len(input_features), hidden_size=500,
                                                 dropout_lstm=0.2).to(DEVICE),
        'model_type': 'handbook'
    },
    { # 6
        'artifact_type': 'model',
        'artifact_name': 'lstm_fit_one_cycle:v0',
        'artifact_path': 'lstm_fit_one_cycle-v0/lstm_fit_one_cycle.pth',
        'model_instance': tsai.all.LSTM(
            c_in=len(input_features),
            c_out=1
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 7
        'artifact_type': 'model',
        'artifact_name': 'fcn_fit_one_cycle:v0',
        'artifact_path': 'fcn_fit_one_cycle-v0/fcn_fit_one_cycle.pth',
        'model_instance': tsai.all.FCN(
            c_in=len(input_features),
            c_out=1
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 8
        'artifact_type': 'model',
        'artifact_name': 'gmlp_fit_one_cycle:v0',
        'artifact_path': 'gmlp_fit_one_cycle-v0/gmlp_fit_one_cycle.pth',
        'model_instance': tsai.all.gMLP(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 9
        'artifact_type': 'model',
        'artifact_name': 'gru_fcn_fit_one_cycle:v0',
        'artifact_path': 'gru_fcn_fit_one_cycle-v0/gru_fcn_fit_one_cycle.pth',
        'model_instance': tsai.all.GRU_FCN(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 10
        'artifact_type': 'model',
        'artifact_name': 'gru_fit_one_cycle:v0',
        'artifact_path': 'gru_fit_one_cycle-v0/gru_fit_one_cycle.pth',
        'model_instance': tsai.all.GRU(
            c_in=len(input_features),
            c_out=1
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 11
        'artifact_type': 'model',
        'artifact_name': 'inceptiontime_fit_one_cycle:v0',
        'artifact_path': 'inceptiontime_fit_one_cycle-v0/inceptiontime_fit_one_cycle.pth',
        'model_instance': tsai.all.InceptionTime(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 12
        'artifact_type': 'model',
        'artifact_name': 'lstm_fcn_fit_one_cycle:v1',
        'artifact_path': 'lstm_fcn_fit_one_cycle-v1/lstm_fcn_fit_one_cycle.pth',
        'model_instance': tsai.all.LSTM_FCN(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 13
        'artifact_type': 'model',
        'artifact_name': 'mlstm_fcn_fit_one_cycle:v0',
        'artifact_path': 'mlstm_fcn_fit_one_cycle-v0/mlstm_fcn_fit_one_cycle.pth',
        'model_instance': tsai.all.MLSTM_FCN(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 14
        'artifact_type': 'model',
        'artifact_name': 'omniscalecnn_fit_one_cycle:v1',
        'artifact_path': 'omniscalecnn_fit_one_cycle-v1/omniscalecnn_fit_one_cycle.pth',
        'model_instance': tsai.all.OmniScaleCNN(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 15
        'artifact_type': 'model',
        'artifact_name': 'rescnn_fit_one_cycle:v1',
        'artifact_path': 'rescnn_fit_one_cycle-v1/rescnn_fit_one_cycle.pth',
        'model_instance': tsai.all.ResCNN(
            c_in=len(input_features),
            c_out=1
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 16
        'artifact_type': 'model',
        'artifact_name': 'resnet_fit_one_cycle:v0',
        'artifact_path': 'resnet_fit_one_cycle-v0/resnet_fit_one_cycle.pth',
        'model_instance': tsai.all.ResNet(
            c_in=len(input_features),
            c_out=1
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 17
        'artifact_type': 'model',
        'artifact_name': 'tsit_fit_one_cycle:v0',
        'artifact_path': 'tsit_fit_one_cycle-v0/tsit_fit_one_cycle.pth',
        'model_instance': tsai.all.TSiT(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN,
            use_token=False
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 18
        'artifact_type': 'model',
        'artifact_name': 'tst_fit_one_cycle:v0',
        'artifact_path': 'tst_fit_one_cycle-v0/tst_fit_one_cycle.pth',
        'model_instance': tsai.all.TST(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    },
    { # 19
        'artifact_type': 'model',
        'artifact_name': 'xcm_fit_one_cycle:v0',
        'artifact_path': 'xcm_fit_one_cycle-v0/xcm_fit_one_cycle.pth',
        'model_instance': tsai.all.XCM(
            c_in=len(input_features),
            c_out=1,
            seq_len=SEQ_LEN
        ).to(DEVICE),
        'model_type': 'tsai'
    }
]

In [ ]:
def pytorch_model_input_preparation(train_df, valid_df, input_features, output_feature, batch_size=64):
    x_train = torch.FloatTensor(train_df[input_features].values)
    x_valid = torch.FloatTensor(valid_df[input_features].values)
    y_train = torch.FloatTensor(train_df[output_feature].values)
    y_valid = torch.FloatTensor(valid_df[output_feature].values)

    training_set = FraudSequenceDataset(x_train, 
                                        y_train,train_df['CUSTOMER_ID'].values, 
                                        train_df['TX_DATETIME'].values,
                                        SEQ_LEN,
                                        padding_mode = "zeros")

    valid_set = FraudSequenceDataset(x_valid, 
                                    y_valid,
                                    valid_df['CUSTOMER_ID'].values, 
                                    valid_df['TX_DATETIME'].values,
                                    SEQ_LEN,
                                    padding_mode = "zeros")

    training_generator, valid_generator = prepare_generators(training_set, valid_set, batch_size=batch_size)

    return training_set, valid_set, training_generator, valid_generator

def tsai_model_input_preparation(train_df, valid_df, input_features, output_feature, batch_size=64):
    x_train, y_train = prepare_sequenced_X_y(train_df, SEQ_LEN, input_features, output_feature)
    x_valid, y_valid = prepare_sequenced_X_y(valid_df, SEQ_LEN, input_features, output_feature)
    X, y, splits = tsai.all.combine_split_data([x_train.numpy(), x_valid.numpy()], [y_train.numpy(), y_valid.numpy()])
    dsets = tsai.all.TSDatasets(X, y, splits=splits, inplace=True)
    dls = tsai.all.TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=batch_size, drop_last=False, device=DEVICE)
    return dsets, dls

In [ ]:
def get_context(outlier, nearest_neighbors, inliers):
    neighbors_index = nearest_neighbors.kneighbors([outlier], return_distance=False).tolist()[0]
    return inliers[neighbors_index]

def replicate_upsampling(outlier, context):
    return np.array([outlier for _ in range(len(context))])

def synthetic_upsampling(C_i_l, o_i, trim_small_values=True): # C_i_l.shape = (x, 5, 15), o_i.shape = (5, 15)
    # transformation matrix for synthetic sampling
    num_c0_new = C_i_l.shape[0] - 1
    coeff_c0_new = np.random.rand(num_c0_new, C_i_l.shape[0]) 

    # use knn to find the closest distance between the outlier and its context
    nbrs_local = KNeighborsTimeSeries(n_neighbors=1).fit(C_i_l) 
    min_dist_to_nbr = nbrs_local.kneighbors([o_i])[0][0, 0]
    min_dist_to_nbr /= C_i_l.shape[2] # normalize distance

    # normalize each row of the coeff_c0_new array so that the sum of each row is equal to 1
    for r in range(coeff_c0_new.shape[0]):
        coeff_c0_new[r, :] /= sum(coeff_c0_new[r, :])

    # flatten last 2 dims of C_i_l otherwise it is impossible to receive insts_c0_new with
    # (cluster_length - 1, 5, 15) shape out of dot product where C_i_l.shape = (cluster_length, 5, 15)
    insts_c0_new = np.dot(coeff_c0_new,
                          C_i_l.reshape(-1, C_i_l.shape[1] * C_i_l.shape[2]) - np.dot(np.ones((C_i_l.shape[0], 1)),
                                                                                      [o_i.flatten()]))

    for r in range(insts_c0_new.shape[0]):                      # shrink to prevent overlap
        insts_c0_new[r, :] *= (0.2 * np.random.rand(1)[0] * min_dist_to_nbr)
    insts_c0_new += np.dot(np.ones((num_c0_new, 1)), [o_i.flatten()])    # origin + shift

    if trim_small_values:
        insts_c0_new[insts_c0_new < 0.001] = 0

    # bring back the 3d shape
    insts_c0 = np.vstack(([o_i], insts_c0_new.reshape(-1, o_i.shape[0], o_i.shape[1])))        

    return insts_c0

def mixed_upsampling(C_i_l, o_i, trim_small_values=True, minimal_outlier_num=4):
    # use ts coin sampling to generate outliers
    O_i_l = synthetic_upsampling(C_i_l, o_i, trim_small_values=trim_small_values)
    # context length must be bigger than number of outliers before SMOTE, otherwise ValueError is raised
    minimal_outlier_num = min(minimal_outlier_num, C_i_l.shape[0] - 1)
    # select minimal number of outliers and include the original outlier at index 0
    outliers_subset = O_i_l[:minimal_outlier_num]
    
    # side note: shrinkage parameter in RandomOversampler does not influence upsampled outliers values
    #
    # k_neighbors must be set to minimal_outlier_num - 1, otherwise ValueError during fit_resample is raised
    ROS = imblearn.over_sampling.SMOTE(sampling_strategy=1, k_neighbors=minimal_outlier_num - 1, random_state=SEED)
    X = np.vstack((outliers_subset.reshape(-1, outliers_subset.shape[1] * outliers_subset.shape[2]),
                   C_i_l.reshape(-1, C_i_l.shape[1] * C_i_l.shape[2])))
    y = np.hstack((
        np.ones((1, outliers_subset.shape[0])),
        np.zeros((1, C_i_l.shape[0]))
    )).T
    X_resampled, Y_resampled = ROS.fit_resample(X, y) # X has to be 2d
    O_i_indices = [i for i in range(len(Y_resampled)) if Y_resampled[i] == 1]
    return X_resampled.reshape(-1, C_i_l.shape[1], C_i_l.shape[2])[O_i_indices]

def cluster(C_i, increase_cluster_num=True):
    kmeans = TimeSeriesKMeans(n_clusters=2, metric='dtw', random_state=SEED).fit(C_i)
    best_silhouette_score = silhouette_score(C_i, kmeans.predict(C_i), metric='dtw')
    L = 2
    for n in range(3, 6):
        new_kmeans = TimeSeriesKMeans(n_clusters=n, metric='dtw', random_state=SEED).fit(C_i)
        new_silhouette_score = silhouette_score(C_i, kmeans.predict(C_i), metric='dtw')
        if new_silhouette_score > best_silhouette_score:
            best_silhouette_score = new_silhouette_score
            kmeans = new_kmeans
            L = n

    if increase_cluster_num:
        L += 1

    C_i_l = np.array([C_i[kmeans.labels_ == i] for i in range(L)])
    return C_i_l    

def abandon_small_clusters(clustered, not_clustered, ratio):
    return np.array([cluster_i for cluster_i in clustered if len(cluster_i) >= ratio * len(not_clustered)])

def create_local_classifier(C_i_l, O_i_l):
    clf = TimeSeriesSVC(kernel='linear', random_state=SEED, gamma=1.) # kernel='gak'
    X = np.vstack((C_i_l, O_i_l))
    y = np.hstack((np.zeros(C_i_l.shape[0]), np.ones(O_i_l.shape[0])))
    clf.fit(X, y)
    return clf

def calculate_all_abnormal_attribute_scores(context_results):
    # LinearTimeSeriesSVC has 75 weights, 1 weight per feature for all sequence elements
    abnormal_attribute_scores = np.zeros(context_results[0]['clf'].svm_estimator_.coef_[0].shape)
    for context_cluster_results in context_results:
        abnormal_attribute_scores += len(context_cluster_results['context_cluster']) * \
            np.abs(context_cluster_results['clf'].svm_estimator_.coef_[0])
    abnormal_attribute_scores /= np.sum([len(context_result['context_cluster']) for context_result in context_results])
    abnormal_attribute_scores /= np.sum(abnormal_attribute_scores)
    return abnormal_attribute_scores

def calculate_outlierness_score(context_results, outlier):
    outlierness_score = 0
    for context_cluster_results in context_results:
        outlierness_score_cluster = abs(context_cluster_results['clf'].decision_function(outlier)) /\
            np.linalg.norm(context_cluster_results['clf'].svm_estimator_.coef_[0], ord=2)
        outlierness_score += np.linalg.norm(len(context_cluster_results['context_cluster']) * \
            outlierness_score_cluster * context_cluster_results['clf'].svm_estimator_.coef_[0] /\
            np.linalg.norm(context_cluster_results['clf'].svm_estimator_.coef_[0]))
    outlierness_score /= sum([len(context_result['context_cluster']) for context_result in context_results])
    return outlierness_score

def tscoin_for_outlier(o_i, inliers, upsampling_technique, abandon_ratio, nearest_neighbors):
    C_i = get_context(o_i, nearest_neighbors, inliers)
    C_i_clusters = cluster(C_i)
    C_i_clusters = abandon_small_clusters(C_i_clusters, C_i, abandon_ratio)
    context_results = []
    for C_i_l in C_i_clusters:
        if upsampling_technique == 'synthetic':
            O_i_l = synthetic_upsampling(C_i_l, o_i)
        elif upsampling_technique == 'replicate':
            O_i_l = replicate_upsampling(o_i, C_i_l)
        else:
            O_i_l = mixed_upsampling(C_i_l, o_i)
        clf = create_local_classifier(C_i_l, O_i_l)
        context_results.append({
            'clf': clf, 
            'context_cluster': C_i_l,
            'upsampled_outlier': O_i_l,
            })
    abnormal_attributes_scores = calculate_all_abnormal_attribute_scores(context_results)
    outlierness_score = calculate_outlierness_score(context_results, o_i.reshape(1, o_i.shape[0], o_i.shape[1]))
    return abnormal_attributes_scores, outlierness_score

def tscoin(inliers, outliers, upsampling_technique='synthetic', abandon_ratio=0.05):
    tscoin_start_time=time.time()
    context_neighbors = int(np.sqrt(len(inliers)))
    nearest_neighbors = KNeighborsTimeSeries(n_neighbors=context_neighbors, metric='dtw') #  metric='euclidean'
    # n_jobs=-1 doesn't seem to decrease execution time
    knn_start_time=time.time()
    # array of shape (n_ts, sz, d) <=> (num_samples, seq_len, num_features)
    nearest_neighbors = nearest_neighbors.fit(inliers)
    knn_end_time = time.time()
    logging.info(f'KNeighbors fitting time: {round(knn_end_time - knn_start_time, 2)} seconds')
    parallel_job_results = Parallel(n_jobs=-2)(delayed(tscoin_for_outlier)(o_i, inliers, upsampling_technique, abandon_ratio,
                                                                           nearest_neighbors) for o_i in outliers)
    tscoin_end_time = time.time()
    outlierness_score_list = [item[0] for item in parallel_job_results]
    abnormal_attributes_scores_list = [item[1] for item in parallel_job_results]
    logging.info(f'TS COIN end, execution time: {round(tscoin_end_time - tscoin_start_time, 2)} seconds')
    return outlierness_score_list, abnormal_attributes_scores_list

In [ ]:
calculate_tscoin_outliers = True
calculate_shap_outliers = True
save_predictions = True
save_explainers_expected_value = True
generate_outliers_summary_plots_flattened = True
generate_outliers_summary_plots_timestep_aggregated = True
calculate_tscoin_inliers_sample = True
calculate_shap_inliers_sample = True

In [ ]:
logging.info('START')
for model_index in tqdm(range(len(models))):
    model_type = models[model_index]['model_type']
    logging.info(f'NEW MODEL: {model_type} {models[model_index]["artifact_name"].split(":")[0]}')
    if model_type == 'handbook':
        torch_train_set, torch_valid_set, torch_train_generator, torch_valid_generator = \
            pytorch_model_input_preparation(train_df, valid_df, input_features, output_feature)
    elif model_type == 'tsai':
        tsai_dsets, tsai_dls = tsai_model_input_preparation(train_df, valid_df, input_features, output_feature)

    # uncomment if 1st time running particular models
    #
    # run = wandb.init()
    # artifact = run.use_artifact(f'mgr-anomaly-tsxai/mgr-anomaly-tsxai-project/{models[model_index]["artifact_name"]}',
    #                             type=models[model_index]['artifact_type'])
    # artifact_dir = artifact.download()
    model = models[model_index]['model_instance']
    model.load_state_dict(torch.load(f'artifacts/{models[model_index]["artifact_path"]}'))
    model.eval()
    logging.info('Weights loaded successfully')
    valid_predictions = get_predictions_sequential(model, torch_valid_generator if model_type == 'handbook'
                                                   else tsai_dls.valid)
    if model_type == 'tsai':
        valid_predictions = torch.nn.Sigmoid()(torch.FloatTensor(valid_predictions)).numpy()
    valid_predictions_int = np.round(valid_predictions).astype(int)
    logging.info('Get predictions finished')
    if save_predictions:
        np.save(
            f'generator_output/outliers_indices/{model_type}/{models[model_index]["artifact_name"].split(":")[0]}.npy',
            np.where(valid_predictions_int == 1)[0]
            )
    if model_type == 'handbook':
        torch_valid_set.output = False
        predicted_inliers = torch_valid_set.features[torch_valid_set.sequences_ids[valid_predictions_int == 0],:]
        predicted_outliers = torch_valid_set.features[torch_valid_set.sequences_ids[valid_predictions_int == 1],:]
        torch_valid_set.output = True
    elif model_type == 'tsai':
        predicted_inliers = tsai_dls.valid.dataset[valid_predictions_int == 0][0]
        predicted_outliers = tsai_dls.valid.dataset[valid_predictions_int == 1][0]
        predicted_inliers = predicted_inliers.reshape(predicted_inliers.shape[0], predicted_inliers.shape[2],
                                                      predicted_inliers.shape[1])
        predicted_outliers = predicted_outliers.reshape(predicted_outliers.shape[0], predicted_outliers.shape[2],
                                                        predicted_outliers.shape[1])
        
    if calculate_tscoin_outliers:
        for upsampling_technique in ['synthetic', 'replicate', 'mixed']:
            logging.info(f'TS COIN on outliers start, upsampling technique: {upsampling_technique}')
            outlierness_score, abnormal_attributes_scores = tscoin(predicted_inliers.numpy(),
                                                                   predicted_outliers.numpy()[:10], upsampling_technique)
            logging.info('TS COIN on outliers finished')
            np.save(
                f'generator_output/outliers_attribute_scores/{model_type}/{upsampling_technique}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.npy',
                np.array(abnormal_attributes_scores)
                )
            np.save(
                f'generator_output/outliers_outlierness_scores/{model_type}/{upsampling_technique}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.npy',
                np.array(outlierness_score)
                )
    else:
        logging.info('TS COIN on outliers skipped')

    if calculate_tscoin_inliers_sample:
        for upsampling_technique in ['synthetic', 'replicate', 'mixed']:
            logging.info(f'TS COIN on inliers sample start, upsampling technique: {upsampling_technique}')
            outlierness_score, abnormal_attributes_scores = tscoin(predicted_inliers[len(predicted_outliers):].numpy(),
                                                                   predicted_inliers[:len(predicted_outliers)].numpy(),
                                                                   upsampling_technique)
            logging.info('TS COIN on inliers sample finished')
            np.save(
                f'generator_output/inliers_attribute_scores/{model_type}/{upsampling_technique}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.npy', 
                np.array(abnormal_attributes_scores)
                )
            np.save(
                f'generator_output/inliers_outlierness_scores/{model_type}/{upsampling_technique}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.npy',
                np.array(outlierness_score))
    else:
        logging.info('TS COIN on inliers sample skipped')

    if calculate_shap_outliers or calculate_shap_inliers_sample or save_explainers_expected_value:
        if model_type == 'handbook':
            _, _, torch_shap_generator, _ = pytorch_model_input_preparation(train_df, valid_df, input_features,
                                                                            output_feature, 1000)
        elif model_type == 'tsai':
            _, tsai_shap_dls = tsai_model_input_preparation(train_df, valid_df, input_features, output_feature, 1000)

        model.train()
        if model_type == 'handbook':
            shap_background_data = next(iter(torch_shap_generator))[0]
        elif model_type == 'tsai':
            shap_background_data = next(iter(tsai_shap_dls.train))[0]
        try:
            explainer = shap.DeepExplainer(model, shap_background_data)
        except Exception as e:
            logging.error('SHAP background data step failed')
            logging.error(f'Exception: {e}')
            logging.info(f'SHAP background data on {shap_background_data.device}')
        else:
            logging.info('SHAP background data step finished')
            if save_explainers_expected_value:
                np.save(
                    f'generator_output/shap_explainer_expected_value/{model_type}/'
                    f'{models[model_index]["artifact_name"].split(":")[0]}.npy',
                    explainer.expected_value
                    )
        if model_type == 'handbook':
            shap_values_dataset = torch_valid_set
        elif model_type == 'tsai':
            shap_values_dataset = tsai_dsets.valid

        if calculate_shap_outliers:
            model.train()
            try:
                outliers_shap_values = explainer.shap_values(predicted_outliers.to(DEVICE)\
                                                             .reshape(predicted_outliers.shape[0], len(input_features),
                                                                      SEQ_LEN))
                clear_output(wait=True)
            except Exception as e:
                logging.error('SHAP values generation on outliers failed')
                logging.error(f'Exception: {e}')
                logging.info(f'SHAP values on {predicted_outliers.to(DEVICE).device}')
            else:
                logging.info('SHAP values generation on outliers finished')
                np.save(
                    f'generator_output/outliers_shap_values/{model_type}/'
                    f'{models[model_index]["artifact_name"].split(":")[0]}.npy', 
                    np.array(outliers_shap_values)
                    )
            model.eval()
        else:
            logging.info('SHAP on outliers skipped')

        if calculate_shap_inliers_sample:
            model.train()
            try:
                inliers_shap_values = explainer.shap_values(predicted_inliers[:len(predicted_outliers)].to(DEVICE)\
                                                            .reshape(predicted_inliers[:len(predicted_outliers)]\
                                                                     .shape[0], len(input_features), SEQ_LEN))
                clear_output(wait=True)
            except Exception as e:
                logging.error('SHAP values generation on inliers sample failed')
                logging.error(f'Exception: {e}')
                logging.info(f'SHAP values on {predicted_inliers.to(DEVICE).device}')
            else:
                logging.info('SHAP values generation on inliers finished')
                np.save(
                    f'generator_output/inliers_shap_values/{model_type}/'
                    f'{models[model_index]["artifact_name"].split(":")[0]}.npy',
                    np.array(inliers_shap_values)
                    )
            model.eval()
        else:
            logging.info('SHAP on inliers sample skipped')
    else:
        logging.info('SHAP on outliers skipped')
        logging.info('SHAP on inliers sample skipped')

    if generate_outliers_summary_plots_flattened:
        try:
            outliers_shap_values = np.load(
                f'generator_output/outliers_shap_values/{model_type}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.npy'
                )
        except Exception as e:
            logging.error('Outliers SHAP values load failed')
            logging.error(f'Exception: {e}')
        else:
            outliers_flattened_shap_values = outliers_shap_values.reshape(outliers_shap_values.shape[0], -1)
            shap.summary_plot(outliers_flattened_shap_values,
                              feature_names=[f'{input_feature} (t-{t})'
                                             for t in range(SEQ_LEN - 1, -1, -1)
                                             for input_feature in input_features],
                                             features=predicted_outliers.reshape(predicted_outliers.shape[0], -1),
                                             max_display=30, sort=True, show=False)
            plt.savefig(
                f'generator_output/outliers_summary_plots_flattened/{model_type}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.png'
                )
            # clear plot each time to prevent overlapping
            plt.clf()
            logging.info('Outliers summary plot flattened generation finished')
    else:
        logging.info('Outliers summary plot flattened skipped')
    
    if generate_outliers_summary_plots_timestep_aggregated:
        try:
            outliers_shap_values = np.load(
                f'generator_output/outliers_shap_values/{model_type}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.npy'
                )
        except Exception as e:
            logging.error('Outliers SHAP values load failed')
            logging.error(f'Exception: {e}')
        else:
            shap_features = np.array([np.sum(predicted_outlier, axis=1) for predicted_outlier in predicted_outliers\
                                      .reshape(predicted_outliers.shape[0], predicted_outliers.shape[2],
                                               predicted_outliers.shape[1]).numpy()])
            outliers_aggregated_shap_values = np.array([np.sum(outlier_shap_values, axis=1) for outlier_shap_values
                                                        in outliers_shap_values])
            shap.summary_plot(outliers_aggregated_shap_values, feature_names=input_features, features=shap_features,
                              show=False)
            plt.savefig(
                f'generator_output/outliers_summary_plots_timestep_aggregated/{model_type}/'
                f'{models[model_index]["artifact_name"].split(":")[0]}.png'
                )
            plt.clf()
            logging.info('Outliers summary plot timestep aggregated generation finished')
    else:
        logging.info('Outliers summary plot timestep aggregated skipped')

logging.info('END')